리트코드 이진 트리 직렬화 & 역직렬화: https://leetcode.com/problems/serialize-and-deserialize-binary-tree/

In [1]:
import collections
import heapq
import functools
import itertools
import re
import sys
import math
import bisect
from typing import *

In [2]:
class TreeNode:
     def __init__(self, val=0, left=None, right=None):
         self.val = val
         self.left = left
         self.right = right

In [3]:
def toBinaryTree(list: List) -> TreeNode:
    size = len(list)
    idx = 1
    
    if size == 0:
        return None
    
    def recursive(i):
        if i > size or list[i - 1] is None:
            return None
        
        node = TreeNode(list[i - 1])
        node.left = recursive(2 * i)
        node.right = recursive(2 * i + 1)
        
        return node
        
    return recursive(idx)

def printTree(root: TreeNode) -> int:
    if root is None:
        print("Tree is empty.")
    
    q = collections.deque([root])
    
    while q:
        for _ in range(len(q)):
            cur = q.popleft()
            if cur.left:
                q.append(cur.left)
            if cur.right:
                q.append(cur.right)
            print(cur.val, end = " ")
        print()

In [4]:
null = None
root = [1,2,3,null,null,4,5]
tree = toBinaryTree(root)

# 직렬화, 역직렬화

이진 트리라는 자료구조는 논리적인 자료 구조이다. 자료 구조나 객체를 파일이나 디스크에 저장하기 위해서 일련의 비트로 바꿔주어야하는데 이를 직렬화 Serialize 라고 한다. 직렬화된 것을 다시 논리적인 형태로 바꾸는 것은 역직렬화 Deserialize라고 한다. <br>
파이썬에서는 pickle이라는 직렬화 모듈을 기본적으로 제공하며 파이썬 객체의 계층 구조를 파일에 저장할 수 있게 바이트 스트림으로 변경하는 작업을 피클링이라고 한다. 이외에도 마샬링 Marshalling이나 플래트닝 Flattening이라고 하기도 한다.

완전 이진 트리 Complete Binary Tree의 번호를 통해 이진 트리의 노드를 배열로 나타낼 수 있다. 비어 있는 위치는 Null로 나타내면 되며 사실상 모든 트리는 효율적이지는 않아도 배열로 나타낼 수 있다.

## 직렬화 

이 문제는 직렬화 시켰다 다시 역직렬화 시키는 문제로 직렬화 알고리즘의 제약이 없기 때문에 BFS로 구현하든 DFS로 구현하든 아무런 상관이 없다. <br>
다만 리트코드에서도 트리를 BFS로 표현하고 이게 번호 순으로 배치되어 직관적이므로 BFS 알고리즘을 사용해본다. <br>

In [5]:
def serialize(root):
        """Encodes a tree to a single string.
        
        :type root: TreeNode
        :rtype: str
        """
        queue = collections.deque([root])
        result = ['#'] # #은 null
        # 트리 BFS 직렬화
        while queue:
            node = queue.popleft()
            if node:
                queue.append(node.left)
                queue.append(node.right)
                
                result.append(str(node.val))
            else:
                result.append('#')
            
        return ' '.join(result) # 리턴 타입이 str이라고 나와 있기 때문에 문자 배열을 스페이스를 구분자로 하여 문자열로 만듦

In [6]:
serialize(tree)

'# 1 2 3 # # 4 5 # # # #'

In [7]:
root

[1, 2, 3, None, None, 4, 5]

첫 #은 배열로 이진 트리를 나타낼 때 노드 번호에 맞추기 위해 보통 0번은 비워두는 것을 나타낸다. <br>
실제로 적용했을 때 리트코드에서 테스트케이스로 가져온 배열로 나타낸 이진 트리와 동일한 것을 확인할 수 있다. <br>
내가 테스트 케이스 용으로 짠 toBinaryTree 함수가 일종의 deserialize 함수라고 보면 될 것 같다.

## 역직렬화

In [8]:
def deserialize(data):
        """Decodes your encoded data to tree.
        
        :type data: str
        :rtype: TreeNode
        """
        # empty 트리 예외처리
        if data == "# #":
            return None
        
        nodes = data.split() # 스페이스로 구분된 문자열을 분리하여 배열로
        
        root = TreeNode(int(nodes[1])) # 1번 노드 값을 이용해 루트 노드 생성
        queue = collections.deque([root])
        index = 2
        
        # 빠른 런너처럼 자식 노드 결과를 먼저 확인 후 큐 삽입
        while queue:
            node = queue.popleft()
            if nodes[index] is not '#':
                node.left = TreeNode(int(nodes[index]))
                queue.append(node.left)
            index += 1
            
            if nodes[index] is not '#':
                node.right = TreeNode(int(nodes[index]))
                queue.append(node.right)
            index += 1
            
        return root

In [9]:
printTree(deserialize(serialize(tree)))

1 
2 3 
4 5 


toBinaryTree 함수를 짤 때 DFS와 BFS를 두고 고민을 했었는데 BFS는 복잡할 것 같아서 DFS를 사용했었다. <br>
이 BFS로 구현된 역직렬화 함수를 보니 DFS로 하기를 진짜 잘 한 것 같다.